##### DNN에 맞춘 데이터 split

In [ ]:
dnn_x_train = air_train.drop(['date', 'tomm_pm'], axis = 1)
dnn_y_train = air_train[['tomm_pm']]

dnn_x_valid = air_valid.drop(['date', 'tomm_pm'], axis = 1)
dnn_y_valid = air_valid[['tomm_pm']]

dnn_x_test = air_test.drop(['date', 'tomm_pm'], axis = 1)
dnn_y_test = air_test[['tomm_pm']]

In [ ]:
#StandardScaler & MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 

scaler.fit(dnn_x_train)

train_scaled = pd.DataFrame(scaler.transform(dnn_x_train))
valid_scaled = pd.DataFrame(scaler.transform(dnn_x_valid))
test_scaled = pd.DataFrame(scaler.transform(dnn_x_test))

In [ ]:
train_scaled.shape, valid_scaled.shape, test_scaled.shape

((8730, 20), (8784, 20), (8760, 20))

##### RNN, LSTM에 맞춘 데이터 변환

In [ ]:
train_x = air_train.drop(['date', 'tomm_pm'], axis = 1)
train_y = air_train[['PM25']]

valid_x = air_valid.drop(['date', 'tomm_pm'], axis = 1)
valid_y = air_valid[['PM25']]

test_x = air_test.drop(['date', 'tomm_pm'], axis = 1)
test_y = air_test[['PM25']]

In [ ]:
train_set = train_x.values
valid_set = valid_x.values
test_set = test_x.values

train_y = train_y.values
valid_y = valid_y.values
test_y = test_y.values

In [ ]:
# data change for LSTM

def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []
    
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
        
    for i in range(start_index, end_index, 24):
        indices = range(i-history_size, i)
        data.append(dataset[indices])
        
        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])
        
    return np.array(data), np.array(labels)

In [ ]:
# 24시간 데이터 이용해 이후 24시간 동안의 PM2.5 예측

BATCH_SIZE = 64
BUFFER_SIZE = 3000
past_history = 24
future_target = 24
STEP = 24

x_train_multi, y_train_multi = multivariate_data(train_set, train_y, 0,
                                                       None, past_history,
                                                       future_target, STEP,
                                                       single_step=False)
x_val_multi, y_val_multi = multivariate_data(valid_set, valid_y,
                                                   0, None, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
x_test_multi, y_test_multi = multivariate_data(test_set, test_y, 0,
                                               None, past_history,
                                               future_target, STEP,
                                               single_step=False)

In [ ]:
# Object to float
x_train_multi = x_train_multi.astype(float)
x_val_multi = x_val_multi.astype(float)
x_test_multi = x_test_multi.astype(float)

y_train_multi = y_train_multi.astype(float)
y_val_multi = y_val_multi.astype(float)
y_test_multi = y_test_multi.astype(float)